In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import pickle



In [2]:
years = list(range(2024,2022,-1))

In [3]:
years

[2024, 2023]

In [4]:
all_matches = []

In [5]:
standings_url = "https://fbref.com/de/wettbewerbe/20/Bundesliga-Statistiken"

In [6]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table') [0]
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/mannschaften/' in l]
    team_urls = [f"http://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com/{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Statistiken", "").replace("-"," ")
        
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match = "Ergebnisse & Spiele")[0]
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l ]
        data = requests.get(f"http://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match = "Torschüsse")[0]
        shooting.columns = shooting.columns.droplevel()
      
        try:
            team_data = matches.merge(shooting[["Datum","Sc", "SaT", "Entf.", "FS","Elf","VeElf"]], on="Datum")
        except ValueError:
            continue
        
        team_data = team_data[team_data["Wett"]== "Bundesliga"]
        team_data ["Saison"] = year
        team_data["Verein"] = team_name
        all_matches.append(team_data)
        time.sleep(1)
    
    

In [7]:
match_df = pd.concat(all_matches)


In [8]:
match_df.columns = [c.lower() for c in match_df.columns]

In [9]:
match_df.to_csv("match.csv")

In [10]:
import pickle

with open('match_df', 'wb') as f:
    pickle.dump(match_df, f)
    